# Setup

In [2]:
import os

from dotenv import load_dotenv
load_dotenv()

azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_KEY")

llm_config = [{"model": azure_deployment,
               "api_type": "azure",
               "temperature": 0.9,
               "api_key": api_key,
               "base_url": azure_endpoint,
               "api_version": "2024-02-15-preview"}] 

# Define Agents

In [10]:
from autogen import UserProxyAgent, AssistantAgent, GroupChat, GroupChatManager
from autogen.coding import LocalCommandLineCodeExecutor

user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
)
engineer = AssistantAgent(
    name="Engineer",
    llm_config={"config_list": llm_config},
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. 
        Wrap the code in a code block that specifies the script type. 
        The user can't modify your code. 
        So do not suggest incomplete code which requires others to modify. 
        Don't use a code block if it's not intended to be executed by the executor.
        Don't include multiple code blocks in one response. 
        Do not ask others to copy and paste the result. 
        Check the execution result returned by the executor.
        If the result indicates there is an error, fix the error and output the code again. 
        Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
    """,
)
scientist = AssistantAgent(
    name="Scientist",
    llm_config={"config_list": llm_config},
    system_message="""Scientist. You follow an approved plan. 
        You are able to categorize posts after seeing their abstracts printed. You don't write code.
    """,
)
planner = AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. 
        Revise the plan based on feedback from admin and critic, until admin approval.
        The plan may involve an engineer who can write code and a scientist who doesn't write codes.
        Explain the plan first. Be clear which step is performed by an engineer and which step is performed by a scientist.
""",
    llm_config={"config_list": llm_config},
)

# Create a local command line code executor.
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir="paper",  # Use the temporary directory to store the code files.
)

executor = UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"executor": executor},  # Use the local command line code executor.
)

critic = AssistantAgent(
    name="Critic",
    system_message="""Critic. Double check plan, claims, code from other agents and provide feedback. 
        Check whether the plan includes adding verifiable info such as source URL. 
        Checks whether plan is actually executed
    """,
    llm_config={"config_list": llm_config},
)

writer = AssistantAgent(
    name="Writer",
    system_message="""Writer. You write a summary of the final output.
    """,
    llm_config={"config_list": llm_config},
)


In [11]:
groupchat = GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor, critic, writer], 
    messages=[], 
    max_round=50,
    send_introductions=True
)
manager = GroupChatManager(groupchat=groupchat, llm_config={"config_list": llm_config})

user_proxy.initiate_chat(
    manager,
    message="""
        Search arxiv.org about the 20 latest papers on AI specifically on Large Language Models. You can categorize the papers based on the subject discussed in the article.
        Possible categories are Retrieval-Augmented Generation, Fine Tuning, Prompt Engineering, Agents
        You need to write code and run the code. Rank the categories based on the number of news in each category. 
        Provide the code and the categories in the response.
        Write a summary of the final output.
    """,
)

Admin (to chat_manager):


        Search arxiv.org about the 20 latest papers on AI specifically on Large Language Models. You can categorize the papers based on the subject discussed in the article.
        Possible categories are Retrieval-Augmented Generation, Fine Tuning, Prompt Engineering, Agents
        You need to write code and run the code. Rank the categories based on the number of news in each category. 
        Provide the code and the categories in the response.
        Write a summary of the final output.
    

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

### Plan

1. **Engineer**: Write a Python script that uses the arXiv API to fetch the 20 latest papers on AI specifically related to Large Language Models.
2. **Engineer**: Print the abstracts of the fetched papers for categorization.
3. **Scientist**: Manually categorize each fetched paper based on the subject discussed in the arti

ChatResult(chat_id=None, chat_history=[{'content': '\n        Search arxiv.org about the 20 latest papers on AI specifically on Large Language Models. You can categorize the papers based on the subject discussed in the article.\n        Possible categories are Retrieval-Augmented Generation, Fine Tuning, Prompt Engineering, Agents\n        You need to write code and run the code. Rank the categories based on the number of news in each category. \n        Provide the code and the categories in the response.\n        Write a summary of the final output.\n    ', 'role': 'assistant', 'name': 'Admin'}, {'content': "Hello everyone. We have assembled a great team today to answer questions and solve tasks. In attendance are:\n\nAdmin: An attentive HUMAN user who can answer questions about the task, and can perform tasks such as running Python code or inputting command line commands at a Linux terminal and reporting back the execution results.\nEngineer: Engineer. You follow an approved plan. Y